In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests


"""1. 定义文件路径和类别"""
file_paths = {
    'Bacteria': 'E:/Python/MI_Analysis/origin_data/心梗组_细菌.xlsx',
    'Fungi': 'E:/Python/MI_Analysis/origin_data/心梗组_真菌.xlsx',
    'Virus': 'E:/Python/MI_Analysis/origin_data/心梗组_病毒.xlsx',
    'Archaea': 'E:/Python/MI_Analysis/origin_data/心梗组_古菌.xlsx'
}

"""2. 定义一个函数来读取和预处理单个文件"""
def read_and_preprocess_data(file_path, domain_name):
    """
    读取一个Excel文件，进行初步清洗和预处理。
    """
    try:
        """读取数据，第一列作为索引"""
        df = pd.read_excel(file_path, index_col=0)
        print(f"{domain_name} 数据维度: {df.shape}")

        """安全转换数值类型，将非数值转换为NaN，然后填充0"""
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        
        """确保没有负值（可选，根据数据情况决定）"""
        df[df < 0] = 0
        
        # """添加类别前缀到特征名，避免后续整合时冲突"""
        # df.index = domain_name + '_' + df.index
        
        return df
    
    except Exception as e:
        print(f"读取 {file_path} 时出错: {e}")
        return None

"""3. 循环读取所有四个文件"""
data_raw = {}
for domain, path in file_paths.items():
    df_domain = read_and_preprocess_data(path, domain)
    if df_domain is not None:
        data_raw[domain] = df_domain

Bacteria 数据维度: (1051, 85)
Fungi 数据维度: (214, 85)
Virus 数据维度: (1443, 85)
Archaea 数据维度: (4639, 85)


In [2]:
"""
4. 确保所有DataFrame的样本（列）一致
提取所有样本名的交集
"""
common_samples = set(data_raw['Bacteria'].columns)
for df in data_raw.values():
    common_samples = common_samples.intersection(df.columns)
common_samples = sorted(common_samples)  # 排序以便保持一致

print(f"共同样本数量: {len(common_samples)}")

"""5. 根据共同样本过滤每个数据集，并确保样本顺序一致"""
for domain in data_raw.keys():
    data_raw[domain] = data_raw[domain][common_samples]

"""
6. 样本分组（从样本名提取，例如CON1为Control，MI1为MI）
创建样本元数据
"""
sample_metadata = pd.DataFrame(index=common_samples)
sample_metadata['Group'] = ['Control' if sample.startswith('CON') else 'MI' for sample in common_samples]
print(sample_metadata['Group'].value_counts())

共同样本数量: 85
Group
Control    47
MI         38
Name: count, dtype: int64


In [14]:
"""7. 基础质控：流行率过滤"""
def prevalence_filter(df, threshold=0.2):
    """
    过滤掉在少于一定比例样本中出现的特征。
    threshold: 流行率阈值，例如0.2表示特征至少在20%的样本中出现。
    """
    prevalence = (df > 0).mean(axis=1)  # 计算每个特征的流行率
    filtered_df = df[prevalence >= threshold]
    print(f"  过滤前: {df.shape[0]} 个特征")
    print(f"  过滤后: {filtered_df.shape[0]} 个特征 (流行率 >= {threshold})")
    return filtered_df

print("\n正在进行流行率过滤 (阈值 0.2)...")
data_filtered = {}
for domain, df in data_raw.items():
    print(f"{domain}:")
    data_filtered[domain] = prevalence_filter(df, threshold=0.2)
    
"""将过滤后的数据保存到Excel的不同Sheet中"""
output_excel_path = 'E:/Python/MI_Analysis/data_figures/filtered_data/filtered_data.xlsx'

"""使用ExcelWriter来写入多个Sheet"""
with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
    for domain, df in data_filtered.items():
        """将每个域的数据写入不同的Sheet，Sheet名称即为域名"""
        df.to_excel(writer, sheet_name=domain)
        print(f"已保存 {domain} 数据到Sheet: {domain}")

print(f"\n所有数据已成功保存到: {output_excel_path}")


正在进行流行率过滤 (阈值 0.2)...
Bacteria:
  过滤前: 1051 个特征
  过滤后: 335 个特征 (流行率 >= 0.2)
Fungi:
  过滤前: 214 个特征
  过滤后: 194 个特征 (流行率 >= 0.2)
Virus:
  过滤前: 1443 个特征
  过滤后: 152 个特征 (流行率 >= 0.2)
Archaea:
  过滤前: 4639 个特征
  过滤后: 365 个特征 (流行率 >= 0.2)
已保存 Bacteria 数据到Sheet: Bacteria
已保存 Fungi 数据到Sheet: Fungi
已保存 Virus 数据到Sheet: Virus
已保存 Archaea 数据到Sheet: Archaea

所有数据已成功保存到: E:/Python/MI_Analysis/data_figures/filtered_data/filtered_data.xlsx


In [9]:
"""Log2FC + P-value + FDR筛选"""
# -----------------------------
# 设置筛选阈值
# -----------------------------
alpha = 0.2          # FDR 和 P-value阈值
log2fc_thresh = 0.3     # Log2FC阈值

# -----------------------------
# 遍历每个类别，计算指标并筛选
# -----------------------------
final_results = {}  # 存放综合筛选结果

for domain, df in data_filtered.items():
    df = df[common_samples]  # 保证样本顺序一致
    results = []

    """---- 计算指标 ----"""
    for feature, abundances in df.iterrows():
        control_abund = abundances[sample_metadata[sample_metadata['Group'] == 'Control'].index]
        mi_abund = abundances[sample_metadata[sample_metadata['Group'] == 'MI'].index]

        """Mann-Whitney U 检验"""
        stat, p_val = mannwhitneyu(control_abund, mi_abund, alternative='two-sided')

        """Log2FC (加极小值 epsilon 避免除零)"""
        epsilon = 1e-6
        mean_control = np.mean(control_abund) + epsilon
        mean_mi = np.mean(mi_abund) + epsilon
        log2fc = np.log2(mean_mi / mean_control)

        results.append({
            'Feature': feature,
            'Log2FC': log2fc,
            'P_value': p_val
        })

    """---- 转 DataFrame 并 FDR 校正 ----"""
    result_df = pd.DataFrame(results).set_index('Feature')
    rejected, fdrs, _, _ = multipletests(result_df['P_value'], alpha=alpha, method='fdr_bh')
    result_df['FDR'] = fdrs

    """保存计算好的指标"""
    result_df.to_excel(f"E:\Python\MI_Analysis\data_figures\domain_specific\Basic_indicator_screening\Plan_3\\{domain}_DifferentialMetrics.xlsx")

    """---- 单指标筛选 ----"""
    df_log2fc = result_df[np.abs(result_df['Log2FC']) >= log2fc_thresh]
    df_log2fc.to_excel(f"E:\Python\MI_Analysis\data_figures\domain_specific\Basic_indicator_screening\Plan_3\\{domain}_Log2FC_filtered.xlsx")
    print(f"{domain} Log2FC 筛选结果: {df_log2fc.shape[0]} 个特征")

    df_pval = result_df[result_df['P_value'] < alpha]
    df_pval.to_excel(f"E:\Python\MI_Analysis\data_figures\domain_specific\Basic_indicator_screening\Plan_3\\{domain}_PValue_filtered.xlsx")
    print(f"{domain} P_value 筛选结果: {df_pval.shape[0]} 个特征")

    df_fdr = result_df[result_df['FDR'] < alpha]
    df_fdr.to_excel(f"E:\Python\MI_Analysis\data_figures\domain_specific\Basic_indicator_screening\Plan_3\\{domain}_FDR_filtered.xlsx")
    print(f"{domain} FDR 筛选结果: {df_fdr.shape[0]} 个特征")

    """---- 综合筛选（Log2FC + P-value + FDR） ----"""
    cond = (np.abs(result_df['Log2FC']) >= log2fc_thresh) & \
           (result_df['P_value'] < alpha) & \
           (result_df['FDR'] < alpha)
    df_final = result_df[cond]
    df_final.to_excel(f"E:\Python\MI_Analysis\data_figures\domain_specific\Basic_indicator_screening\Plan_3\\{domain}_FinalFiltered.xlsx")
    final_results[domain] = df_final
    print(f"{domain} 综合筛选显著特征: {df_final.shape[0]} 个")

# -----------------------------
# 所有微生物类别整合到一个 Excel
# -----------------------------
with pd.ExcelWriter('E:\Python\MI_Analysis\data_figures\domain_specific\Basic_indicator_screening\Plan_3\\AllDomains_FinalFiltered.xlsx') as writer:
    for domain, df in final_results.items():
        df.to_excel(writer, sheet_name=domain[:30])

print("\n全部计算和筛选完成，结果已保存！")

Bacteria Log2FC 筛选结果: 254 个特征
Bacteria P_value 筛选结果: 108 个特征
Bacteria FDR 筛选结果: 17 个特征
Bacteria 综合筛选显著特征: 17 个
Fungi Log2FC 筛选结果: 83 个特征
Fungi P_value 筛选结果: 52 个特征
Fungi FDR 筛选结果: 21 个特征
Fungi 综合筛选显著特征: 21 个
Virus Log2FC 筛选结果: 139 个特征
Virus P_value 筛选结果: 50 个特征
Virus FDR 筛选结果: 11 个特征
Virus 综合筛选显著特征: 9 个
Archaea Log2FC 筛选结果: 295 个特征
Archaea P_value 筛选结果: 95 个特征
Archaea FDR 筛选结果: 0 个特征
Archaea 综合筛选显著特征: 0 个

全部计算和筛选完成，结果已保存！


In [11]:
"""Log2FC + FDR筛选"""
# -----------------------------
# 设置筛选阈值
# -----------------------------
alpha = 0.2          # FDR 阈值
log2fc_thresh = 0.7  # Log2FC阈值

# -----------------------------
# 遍历每个域，计算指标并筛选
# -----------------------------
final_results = {}  # 存放综合筛选结果

for domain, df in data_filtered.items():
    df = df[common_samples]  # 保证样本顺序一致
    results = []

    """---- 1. 计算指标 ----"""
    for feature, abundances in df.iterrows():
        control_abund = abundances[sample_metadata[sample_metadata['Group'] == 'Control'].index]
        mi_abund = abundances[sample_metadata[sample_metadata['Group'] == 'MI'].index]

        """Mann-Whitney U 检验"""
        stat, p_val = mannwhitneyu(control_abund, mi_abund, alternative='two-sided')

        """Log2FC (加极小值 epsilon 避免除零)"""
        epsilon = 1e-6
        mean_control = np.mean(control_abund) + epsilon
        mean_mi = np.mean(mi_abund) + epsilon
        log2fc = np.log2(mean_mi / mean_control)

        results.append({
            'Feature': feature,
            'Log2FC': log2fc,
            'P_value': p_val
        })

    """---- 转 DataFrame 并 FDR 校正 ----"""
    result_df = pd.DataFrame(results).set_index('Feature')
    rejected, fdrs, _, _ = multipletests(result_df['P_value'], alpha=alpha, method='fdr_bh')
    result_df['FDR'] = fdrs

    """保存计算好的指标"""
    result_df.to_excel(f"E:/Python/MI_Analysis/data_figures/domain_specific/Basic_indicator_screening/Plan_5/{domain}_DifferentialMetrics.xlsx")

    """---- 单指标筛选 ----"""
    df_log2fc = result_df[np.abs(result_df['Log2FC']) >= log2fc_thresh]
    df_log2fc.to_excel(f"E:/Python/MI_Analysis/data_figures/domain_specific/Basic_indicator_screening/Plan_5/{domain}_Log2FC_filtered.xlsx")
    print(f"{domain} Log2FC 筛选结果: {df_log2fc.shape[0]} 个特征")

    df_fdr = result_df[result_df['FDR'] < alpha]
    df_fdr.to_excel(f"E:/Python/MI_Analysis/data_figures/domain_specific/Basic_indicator_screening/Plan_5/{domain}_FDR_filtered.xlsx")
    print(f"{domain} FDR 筛选结果: {df_fdr.shape[0]} 个特征")

    """---- 综合筛选（Log2FC + FDR） ----"""
    cond = (np.abs(result_df['Log2FC']) >= log2fc_thresh) & (result_df['FDR'] < alpha)
    df_final = result_df[cond]
    df_final.to_excel(f"E:/Python/MI_Analysis/data_figures/domain_specific/Basic_indicator_screening/Plan_5/{domain}_FinalFiltered.xlsx")
    final_results[domain] = df_final
    print(f"{domain} 综合筛选显著特征: {df_final.shape[0]} 个")

# -----------------------------
# 所有域整合到一个 Excel
# -----------------------------
with pd.ExcelWriter('E:/Python/MI_Analysis/data_figures/domain_specific/Basic_indicator_screening/Plan_5/AllDomains_FinalFiltered.xlsx') as writer:
    for domain, df in final_results.items():
        df.to_excel(writer, sheet_name=domain[:30])

print("\n全部计算和筛选完成，结果已保存！")

Bacteria Log2FC 筛选结果: 166 个特征
Bacteria FDR 筛选结果: 17 个特征
Bacteria 综合筛选显著特征: 15 个
Fungi Log2FC 筛选结果: 25 个特征
Fungi FDR 筛选结果: 21 个特征
Fungi 综合筛选显著特征: 11 个
Virus Log2FC 筛选结果: 103 个特征
Virus FDR 筛选结果: 11 个特征
Virus 综合筛选显著特征: 8 个
Archaea Log2FC 筛选结果: 184 个特征
Archaea FDR 筛选结果: 0 个特征
Archaea 综合筛选显著特征: 0 个

全部计算和筛选完成，结果已保存！


In [13]:
"""Log2FC + P_value筛选古菌、Log2FC + FDR筛选其它"""
# -----------------------------
# 设置筛选阈值
# -----------------------------
log2fc_thresh = 0.7    # Log2FC阈值
alpha_pval = 0.05      # 古菌 P-value 阈值
alpha_fdr = 0.2        # 非古菌 FDR阈值

# -----------------------------
# 遍历每个域，计算指标并筛选
# -----------------------------
final_results = {}  # 存放综合筛选结果

for domain, df in data_filtered.items():
    df = df[common_samples]  # 保证样本顺序一致
    results = []

    """---- 计算指标 ----"""
    for feature, abundances in df.iterrows():
        control_abund = abundances[sample_metadata[sample_metadata['Group'] == 'Control'].index]
        mi_abund = abundances[sample_metadata[sample_metadata['Group'] == 'MI'].index]

        """Mann-Whitney U 检验"""
        stat, p_val = mannwhitneyu(control_abund, mi_abund, alternative='two-sided')

        """Log2FC (加极小值 epsilon 避免除零)"""
        epsilon = 1e-6
        mean_control = np.mean(control_abund) + epsilon
        mean_mi = np.mean(mi_abund) + epsilon
        log2fc = np.log2(mean_mi / mean_control)

        results.append({
            'Feature': feature,
            'Log2FC': log2fc,
            'P_value': p_val
        })

    """---- 转 DataFrame 并 FDR 校正 ----"""
    result_df = pd.DataFrame(results).set_index('Feature')
    rejected, fdrs, _, _ = multipletests(result_df['P_value'], alpha=alpha_fdr, method='fdr_bh')
    result_df['FDR'] = fdrs

    """保存计算好的指标"""
    result_df.to_excel(f"E:\\Python\\MI_Analysis\\data_figures\\domain_specific\\Basic_indicator_screening\\Plan_6\\{domain}_DifferentialMetrics.xlsx")

    """---- 单指标筛选 ----"""
    df_log2fc = result_df[np.abs(result_df['Log2FC']) >= log2fc_thresh]
    df_log2fc.to_excel(f"E:\\Python\\MI_Analysis\\data_figures\\domain_specific\\Basic_indicator_screening\\Plan_6\\{domain}_Log2FC_filtered.xlsx")
    print(f"{domain} Log2FC 筛选结果: {df_log2fc.shape[0]} 个特征")

    df_pval = result_df[result_df['P_value'] < alpha_pval]
    df_pval.to_excel(f"E:\\Python\\MI_Analysis\\data_figures\\domain_specific\\Basic_indicator_screening\\Plan_6\\{domain}_PValue_filtered.xlsx")
    print(f"{domain} P_value 筛选结果: {df_pval.shape[0]} 个特征")

    df_fdr = result_df[result_df['FDR'] < alpha_fdr]
    df_fdr.to_excel(f"E:\\Python\\MI_Analysis\\data_figures\\domain_specific\\Basic_indicator_screening\\Plan_6\\{domain}_FDR_filtered.xlsx")
    print(f"{domain} FDR 筛选结果: {df_fdr.shape[0]} 个特征")

    """---- 综合筛选 ----"""
    if domain == 'Archaea':
        """古菌只用 Log2FC + P_value"""
        cond = (np.abs(result_df['Log2FC']) >= log2fc_thresh) & (result_df['P_value'] < alpha_pval)
    else:
        """其他微生物用 Log2FC + FDR"""
        cond = (np.abs(result_df['Log2FC']) >= log2fc_thresh) & (result_df['FDR'] < alpha_fdr)

    df_final = result_df[cond]
    df_final.to_excel(f"E:\\Python\\MI_Analysis\\data_figures\\domain_specific\\Basic_indicator_screening\\Plan_6\\{domain}_FinalFiltered.xlsx")
    final_results[domain] = df_final
    print(f"{domain} 综合筛选显著特征: {df_final.shape[0]} 个")

# -----------------------------
# 所有域整合到一个 Excel
# -----------------------------
with pd.ExcelWriter('E:\\Python\\MI_Analysis\\data_figures\\domain_specific\\Basic_indicator_screening\\Plan_6\\AllDomains_FinalFiltered.xlsx') as writer:
    for domain, df in final_results.items():
        df.to_excel(writer, sheet_name=domain[:30])

print("\n全部计算和筛选完成，结果已保存！")


Bacteria Log2FC 筛选结果: 166 个特征
Bacteria P_value 筛选结果: 50 个特征
Bacteria FDR 筛选结果: 17 个特征
Bacteria 综合筛选显著特征: 15 个
Fungi Log2FC 筛选结果: 25 个特征
Fungi P_value 筛选结果: 27 个特征
Fungi FDR 筛选结果: 21 个特征
Fungi 综合筛选显著特征: 11 个
Virus Log2FC 筛选结果: 103 个特征
Virus P_value 筛选结果: 16 个特征
Virus FDR 筛选结果: 11 个特征
Virus 综合筛选显著特征: 8 个
Archaea Log2FC 筛选结果: 184 个特征
Archaea P_value 筛选结果: 37 个特征
Archaea FDR 筛选结果: 0 个特征
Archaea 综合筛选显著特征: 19 个

全部计算和筛选完成，结果已保存！
